## Cell Counting

This notebook provides you with a complete code example that counts the cells in a microscopic image using a U-Net.

## Loading the Dataset

This is the BBBC039v1 dataset by Caicedo et al. 2018, available from the Broad Bioimage Benchmark Collection https://data.broadinstitute.org/bbbc/BBBC039/ (Ljosa et al., Nature Methods, 2012).

This data set has a total of 200 fields of view of nuclei captured with fluorescence microscopy using the Hoechst stain. The collection has around 23,000 single nuclei manually annotated to establish a ground truth collection for segmentation evaluation.

The images are stored as TIFF files with 520 x 696 pixels at 16 bits. Ground truth annotations are stored as PNG files encoding masks of independent nuclei.

In [ ]:
import os
import deeptrack as dt

if not os.path.exists("cell_counting_dataset"):
    os.system("git clone https://github.com/DeepTrackAI/cell_counting_dataset")

image_paths = dt.sources.ImageFolder(
    root=os.path.join("cell_counting_dataset", "cell nuclei", "images")
)
label_paths = dt.sources.ImageFolder(
    root=os.path.join("cell_counting_dataset", "cell nuclei", "masks")
)
sources = dt.sources.Source(image=image_paths, label=label_paths)

## Creating a Pipeline to Load the Images and Segmentation Masks

Create a pipeline to load the images and labels ...

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader

im_pipeline = (dt.LoadImage(sources.image.path) >> dt.Divide(3000)
               >> dt.Clip(0, 1) >> dt.AsType("float"))
lab_pipeline = dt.LoadImage(sources.label.path)[..., :1] >> dt.AsType("float")

pipeline = (
    (im_pipeline & lab_pipeline)
    >> dt.Crop(crop=(512, 688, None), corner=(0, 0))
    >> dt.MoveAxis(2, 0)
    >> dt.pytorch.ToTensor(dtype=torch.float)
)

test_dataset = dt.pytorch.Dataset(pipeline, sources)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

... count cells using connected components ...

In [ ]:
from skimage import morphology as skmorph

image, mask = test_dataset[np.random.choice(len(test_dataset))]
labeled_mask = skmorph.label(mask)
number_of_cells = labeled_mask.max()

... and visualize the data.

In [ ]:
from matplotlib import pyplot as plt

fig, axs = plt.subplots(1, 3, figsize=(10, 4))

axs[0].imshow(image.permute(1, 2, 0), vmin=0, vmax=1, cmap="gray")

axs[1].imshow(mask.permute(1, 2, 0), vmin=0, vmax=1, cmap="gray")
axs[1].axis("off")
axs[1].set_title(f"Number of cells = {number_of_cells}")

axs[2].hist(np.array(image).flatten(), bins=200, range=(0, 1))
axs[2].set_xlim([0, 1])
axs[2].set_xlabel("Pixel Intensity")
axs[2].set_yscale("log")
axs[2].set_ylabel("Log Frequency")
axs[2].set_title("Pixel Intensity Distribution")

plt.tight_layout()
plt.show()

## Simulating the Cell Images and Segmentation Masks

### Simulating Images of Cell Nuclei

Simulate a single cell nucleus as an ellipse ...

In [ ]:
training_image_size = 256

def random_ellipse_radius():
    """Return the three axes of an ellipse."""
    desired_ellipse_area = (np.random.uniform(3, 4)) ** 2
    desired_radius_ratio = np.random.uniform(1, 1.5)
    major_axis_radius = np.sqrt(desired_ellipse_area) * desired_radius_ratio
    minor_axis_radius = np.sqrt(desired_ellipse_area) / desired_radius_ratio
    z_radius = np.sqrt(desired_ellipse_area) * np.random.uniform(0.2, 0.4)
    return (major_axis_radius, minor_axis_radius, z_radius) * dt.units.um

ellipse = dt.Ellipsoid(
    radius=random_ellipse_radius,
    intensity=lambda: np.random.uniform(0.5, 1.5),
    position=lambda: np.random.uniform(5, training_image_size - 5, size=2),
    rotation=lambda: np.random.uniform(0, 2 * np.pi),
)
optics = dt.Fluorescence(
    resolution=1e-6,
    magnification=6,
    wavelength=400e-9,
    NA=lambda: np.random.uniform(0.9, 1.1),
    output_region=(0, 0, training_image_size, training_image_size),
)
image_pipeline = optics(ellipse)

fig, axs = plt.subplots(1, 5, figsize=(10, 2))
for ax in axs:
    image_pipeline.update()
    image = image_pipeline()
    ax.imshow(image, cmap="gray")
    ax.axis("off")
plt.tight_layout()
plt.show()

... simulate multiple nuclei as ellipses with elastic deformations ...

In [ ]:
training_image_size = 256

def random_ellipse_radius():
    """Return the three axes of an ellipse."""
    desired_ellipse_area = (np.random.uniform(3, 4)) ** 2
    desired_radius_ratio = np.random.uniform(1, 1.5)
    major_axis_radius = np.sqrt(desired_ellipse_area) * desired_radius_ratio
    minor_axis_radius = np.sqrt(desired_ellipse_area) / desired_radius_ratio
    z_radius = np.sqrt(desired_ellipse_area) * np.random.uniform(0.2, 0.4)
    return (major_axis_radius, minor_axis_radius, z_radius) * dt.units.um

ellipse = dt.Ellipsoid(
    radius=random_ellipse_radius,
    intensity=lambda: np.random.uniform(0.5, 1.5),
    position=lambda: np.random.uniform(5, training_image_size - 5, size=2),
    rotation=lambda: np.random.uniform(0, 2 * np.pi),
)
synthetic_nuclei = (
    (ellipse ^ (lambda: np.random.randint(5, 10)))
    >> dt.Pad(px=(10, 10, 10, 10), keep_size=False)
    >> dt.ElasticTransformation(alpha=100, sigma=10, order=1)
    >> dt.CropTight()
)
optics = dt.Fluorescence(
    resolution=1e-6,
    magnification=6,
    wavelength=400e-9,
    NA=lambda: np.random.uniform(0.9, 1.1),
    output_region=(0, 0, training_image_size, training_image_size),
)
image_pipeline = optics(synthetic_nuclei)

fig, axs = plt.subplots(1, 5, figsize=(10, 2))
for ax in axs:
    image_pipeline.update()
    image = image_pipeline()
    ax.imshow(image, cmap="gray")
    ax.axis("off")
plt.tight_layout()
plt.show()

... simulate multiple nuclei as ellipses with elastic deformations and noise ...

In [ ]:
training_image_size = 256

def random_ellipse_radius():
    """Return the three axes of an ellipse."""
    desired_ellipse_area = (np.random.uniform(3, 4)) ** 2
    desired_radius_ratio = np.random.uniform(1, 1.5)
    major_axis_radius = np.sqrt(desired_ellipse_area) * desired_radius_ratio
    minor_axis_radius = np.sqrt(desired_ellipse_area) / desired_radius_ratio
    z_radius = np.sqrt(desired_ellipse_area) * np.random.uniform(0.2, 0.4)
    return (major_axis_radius, minor_axis_radius, z_radius) * dt.units.um

ellipse = dt.Ellipsoid(
    radius=random_ellipse_radius,
    intensity=lambda: np.random.uniform(0.5, 1.5),
    position=lambda: np.random.uniform(5, training_image_size - 5, size=2),
    rotation=lambda: np.random.uniform(0, 2 * np.pi),
)
synthetic_nuclei = (
    (ellipse ^ (lambda: np.random.randint(5, 10)))
    >> dt.Pad(px=(10, 10, 10, 10), keep_size=False)
    >> dt.ElasticTransformation(alpha=100, sigma=10, order=1)
    >> dt.CropTight()
)
synthetic_nuclei_mask = synthetic_nuclei > 0
long_range_noise = (
    synthetic_nuclei
    >> dt.Poisson(snr=lambda: 0.2)
    >> dt.GaussianBlur(sigma=3.5)
)
short_range_noise = (
    synthetic_nuclei
    >> dt.Poisson(snr=lambda: 1.0)
    >> dt.GaussianBlur(sigma=1.5)
)
random_range_noise = (
    synthetic_nuclei
    >> dt.Poisson(snr=lambda: np.random.uniform(0.5, 1.5))
    >> dt.GaussianBlur(sigma=lambda: np.random.uniform(0.75, 1.5))
)
noisy_synthetic_nuclei = synthetic_nuclei_mask * (
    long_range_noise + short_range_noise + random_range_noise
) / 3
optics = dt.Fluorescence(
    resolution=1e-6,
    magnification=6,
    wavelength=400e-9,
    NA=lambda: np.random.uniform(0.9, 1.1),
    output_region=(0, 0, training_image_size, training_image_size),
)
image_pipeline = optics(noisy_synthetic_nuclei)

fig, axs = plt.subplots(1, 5, figsize=(10, 2))
for ax in axs:
    image_pipeline.update()
    image = image_pipeline()
    ax.imshow(image, cmap="gray")
    ax.axis("off")
plt.tight_layout()
plt.show()

... simulate multiple nuclei as non-overlapping ellipses with elastic deformations and noise ...

In [ ]:
training_image_size = 256

def random_ellipse_radius():
    """Return the three axes of an ellipse."""
    desired_ellipse_area = (np.random.uniform(3, 4)) ** 2
    desired_radius_ratio = np.random.uniform(1, 1.5)
    major_axis_radius = np.sqrt(desired_ellipse_area) * desired_radius_ratio
    minor_axis_radius = np.sqrt(desired_ellipse_area) / desired_radius_ratio
    z_radius = np.sqrt(desired_ellipse_area) * np.random.uniform(0.2, 0.4)
    return (major_axis_radius, minor_axis_radius, z_radius) * dt.units.um

ellipse = dt.Ellipsoid(
    radius=random_ellipse_radius,
    intensity=lambda: np.random.uniform(0.5, 1.5),
    position=lambda: np.random.uniform(5, training_image_size - 5, size=2),
    rotation=lambda: np.random.uniform(0, 2 * np.pi),
)
synthetic_nuclei = (
    (ellipse ^ (lambda: np.random.randint(5, 10)))
    >> dt.Pad(px=(10, 10, 10, 10), keep_size=False)
    >> dt.ElasticTransformation(alpha=100, sigma=10, order=1)
    >> dt.CropTight()
)
synthetic_nuclei_mask = synthetic_nuclei > 0
long_range_noise = (
    synthetic_nuclei
    >> dt.Poisson(snr=lambda: 0.2)
    >> dt.GaussianBlur(sigma=3.5)
)
short_range_noise = (
    synthetic_nuclei
    >> dt.Poisson(snr=lambda: 1.0)
    >> dt.GaussianBlur(sigma=1.5)
)
random_range_noise = (
    synthetic_nuclei
    >> dt.Poisson(snr=lambda: np.random.uniform(0.5, 1.5))
    >> dt.GaussianBlur(sigma=lambda: np.random.uniform(0.75, 1.5))
)
noisy_synthetic_nuclei = synthetic_nuclei_mask * (
    long_range_noise + short_range_noise + random_range_noise
) / 3
non_overlap_nuclei = dt.NonOverlapping(noisy_synthetic_nuclei, min_distance=-6)
optics = dt.Fluorescence(
    resolution=1e-6,
    magnification=6,
    wavelength=400e-9,
    NA=lambda: np.random.uniform(0.9, 1.1),
    output_region=(0, 0, training_image_size, training_image_size),
)
image_pipeline = (optics(non_overlap_nuclei)
                  >> dt.Gaussian(sigma=lambda: np.random.uniform(0, 0.1))
                  >> dt.Divide(lambda: np.random.uniform(14, 20))
                  >> dt.Add(lambda: np.random.uniform(-0.05, 0.15))
                  >> dt.Clip(0, 1)
                  >> dt.AsType("float"))

fig, axs = plt.subplots(1, 5, figsize=(10, 2))
for ax in axs:
    image_pipeline.update()
    image = image_pipeline()
    ax.imshow(image, cmap="gray")
    ax.axis("off")
plt.tight_layout()
plt.show()

### Simulating the Segmentation Masks

In [ ]:
from skimage.morphology import isotropic_erosion

def get_mask(radius):
    """Apply isotropic erosion to a binary mask."""
    def inner(mask):
        mask = np.sum(mask, -1, keepdims=True) > 0
        mask = np.pad(mask, [(1, 1), (1, 1), (0, 0)], mode="constant")
        mask = isotropic_erosion(mask, radius=radius)
        mask = mask[1:-1, 1:-1]
        return mask
    return inner

label_pipeline = (
    non_overlap_nuclei
    >> dt.SampleToMasks(get_mask, radius=6, output_region=optics.output_region,
                        merge_method="or")
    >> dt.AsType("float")
)

### Combining the Image and Segmentation Pipelines

In [ ]:
image_and_gt_pipeline = (
    (image_pipeline & label_pipeline)
    >> dt.MoveAxis(2, 0)
    >> dt.pytorch.ToTensor(dtype=torch.float)
)

train_dataset = dt.pytorch.Dataset(image_and_gt_pipeline, length=640,
                                   replace=0.01)

### Visualizing a simulated image and relative segmentation mask

In [ ]:
image, mask = train_dataset[np.random.choice(len(train_dataset))]

fig, axs = plt.subplots(1, 3, figsize=(10, 4))

axs[0].imshow(image.permute(1, 2, 0), vmin=0, vmax=1, cmap="gray")
axs[0].set_title("Simulated Image")

axs[1].imshow(mask.permute(1, 2, 0), vmin=0, vmax=1, cmap="gray")
axs[1].axis("off")
axs[1].set_title("Simulated Mask")

axs[2].hist(np.array(image).flatten(), bins=200, range=(0, 1))
axs[2].set_xlim([0, 1])
axs[2].set_xlabel("Pixel Intensity")
axs[2].set_yscale("log")
axs[2].set_ylabel("Log Frequency")
axs[2].set_title("Pixel Intensity Distribution")

plt.tight_layout()
plt.show()

## Implementing and Training the U-Net

Instantiate the U-Net ...

In [ ]:
import deeplay as dl

unet = dl.UNet2d(in_channels=1, channels=[16, 32, 64], out_channels=1)

... compile it ...

In [ ]:
from torch.nn import BCEWithLogitsLoss

unet_regressor = dl.Regressor(model=unet, loss=BCEWithLogitsLoss(), 
                              optimizer=dl.Adam()).create()


... and train it.

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
unet_trainer = dl.Trainer(max_epochs=200, accelerator="auto")
unet_trainer.fit(unet_regressor, train_loader)

## Testing the Trained U-Net on Simulated Data

In [ ]:
image_and_gt_pipeline.update()
image, mask = image_and_gt_pipeline()
pred = unet_regressor(image.unsqueeze(0)).sigmoid().detach()

fig, axs = plt.subplots(1, 3, figsize=(10, 4))

axs[0].imshow(image.squeeze(), vmin=0, vmax=1, cmap="gray")
axs[0].set_title("Simulated Image")

axs[1].imshow(mask.squeeze(), vmin=0, vmax=1, cmap="gray")
axs[1].axis("off")
axs[1].set_title("Simulated Mask")

axs[2].imshow(pred.squeeze(), cmap="gray")
axs[2].axis("off")
axs[2].set_title("Prediction")

plt.tight_layout()
plt.show()

## Testing on Experimental Data

Calculate the cell counts using the labeled and predicted segmentation masks ...

In [ ]:
images, labels, preds = [], [], []
for image, label in test_loader:
    images.append(image)
    labels.append(label)
    pred = unet_regressor(image).detach()
    pred = torch.nn.functional.sigmoid(pred).cpu()
    preds.append(pred)
images = torch.cat(images, dim=0)
labels = torch.cat(labels, dim=0)
preds = torch.cat(preds, dim=0)

true_count = np.array([skmorph.label(l.squeeze()).max() for l in labels])
pred_count = np.array([skmorph.label(p.squeeze() > .995).max() for p in preds])

mae = abs(pred_count - true_count).mean()
mpe = (abs(pred_count[true_count > 0] - true_count[true_count > 0])
       / (true_count[true_count > 0])).mean()

print(f"MAE: {mae:.2f} MPE: {mpe:.2f}")

... plot an image of cells with labeled and predicted segmentations ...

In [ ]:
i = np.random.choice(len(images))

fig, axs = plt.subplots(1, 3, figsize=(10, 4))

axs[0].imshow(images[i].squeeze(), vmin=0, vmax=1, cmap="gray")
axs[0].set_title("Image")

axs[1].imshow(labels[i].squeeze(), vmin=0, vmax=1, cmap="gray")
axs[1].axis("off")
axs[1].set_title(f"Label - {true_count[i]} cells")

axs[2].imshow(preds[i].squeeze(), cmap="gray")
axs[2].axis("off")
axs[2].set_title(f"Prediction - {pred_count[i]} cells")

plt.tight_layout()
plt.show()

... and plot the predicted cell counts vs. the true cell counts.

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(true_count, pred_count, alpha=0.75)
plt.axline([0, 0], [1, 1], color="black")
plt.xlabel("True Count")
plt.ylabel("Predicted Count")
plt.xlim(0, 250)
plt.ylim(0, 250)
plt.show()